In [1]:
import os
import sys
import numpy as np
import pandas as pd
from Utils.forest_cover import get_cover,green_cover_plot, green_cover_plot_smooth
import cv2
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from PIL import Image, ImageFilter
import imutils
from pyimagesearch.colorlabeler import ColorLabeler
from Utils.read_coord import kml_coord
from Utils.sentinal_request import senti_api,bbox_cord
import matplotlib.pyplot as plt


C:\Users\DELL\anaconda3\envs\sentinal\lib\site-packages\fastkml\config.py:28: UserWarning: Package `lxml` missing. Pretty print will be disabled
  warnings.warn("Package `lxml` missing. Pretty print will be disabled")
C:\Users\DELL\anaconda3\envs\sentinal\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
file_path = input('Enter kml file path: ')
print(file_path)
#C:\Users\Harith\EngenuityAI\SPE\SPE-development\sentinal\gAMPOLA TEST SITE 2.kml
if os.path.exists(file_path):
    print('The file exists')

    with open(file_path, 'r', encoding='utf-8-sig') as f:
        lines = f.readlines()

        #print(lines)
else:
    print('The specified file does NOT exist')



# getting polygon coordinates
poly_coord = kml_coord(file_path)
#test_kml_files/BorealForest_Canada.kml

Enter kml file path: test_kml_files/BorealForest_Canada.kml
test_kml_files/BorealForest_Canada.kml
The file exists


In [5]:
#for original
bbx_coord,poly = bbox_cord(poly_coord)

In [6]:
img = cv2.imread('output/gren_cover_no_legendBorealForest_Canada.png')
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

In [7]:
#img = np.flip(img,0)
#hsv = np.flip(hsv,0)

In [8]:
#define colours

#green color 
lower_green = np.array([40,70,80])
upper_green = np.array([70,255,255])

#white
lower_white = np.array([255,255,255])
upper_white = np.array([255,255,255])


#add col mask
green = cv2.inRange(hsv, lower_green, upper_green)
white = cv2.inRange(img, lower_white, upper_white)#use bgr instead of img

In [9]:
def get_contours(img,hsv):
    #add col mask
    green = cv2.inRange(hsv, lower_green, upper_green)
    white = cv2.inRange(img, lower_white, upper_white)#use bgr instead of img
    #gree conoturs
    cnts_green = cv2.findContours(green,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cnts_green = imutils.grab_contours(cnts_green)
    #white contours
    cnts_white = cv2.findContours(white,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cnts_white = imutils.grab_contours(cnts_white)
    
    return cnts_green,cnts_white[1] #the boundry contour

In [10]:
def point_to_coord_boundry(cnt,bbx_coord):
    poly_arr = np.squeeze(cnt,1)
    #get the bbox
    df = pd.DataFrame(poly_arr, columns = ['lat','long'])
    Xmin = df['lat'].min()
    Ymin = df['long'].min()
    Xmax = df['lat'].max()
    Ymax = df['long'].max()
    bbox = [Xmin,Ymin,Xmax,Ymax]
    
    #coordinate lengths
    coord_len = bbx_coord[2] - bbx_coord[0]
    coord_h = bbx_coord[3] - bbx_coord[1]
    
    #array point lengths
    img_len = bbox[2] - bbox[0]
    img_h = bbox[3] - bbox[1]
    
    #calculate ration diff
    ratio_len = coord_len/img_len
    ratio_h = coord_h/img_h
    
    #get the conversion 
    #coord = point min + (point-point min) * ratio
    lat = []
    long = []
    for index, row in df.iterrows():
        x = (row['lat'] - bbox[0]) * ratio_len
        y = (row['long'] - bbox[1]) * ratio_h
        #print(x)
        lat.append(bbx_coord[0]+x)
        long.append(bbx_coord[3]-y)
        
    poly = list(zip(lat, long))
    return poly,bbox

In [11]:
def point_to_coord_green(cnt,bbx_coord,bbox):
    poly_arr = np.squeeze(cnt,1)
    #get the bbox
    df = pd.DataFrame(poly_arr, columns = ['lat','long'])

    #bbox = [Xmin,Ymin,Xmax,Ymax]
    
    #coordinate lengths
    coord_len = bbx_coord[2] - bbx_coord[0]
    coord_h = bbx_coord[3] - bbx_coord[1]
    
    #array point lengths
    img_len = bbox[2] - bbox[0]
    img_h = bbox[3] - bbox[1]
    
    #calculate ration diff
    ratio_len = coord_len/img_len
    ratio_h = coord_h/img_h

    #bbox,ratio_len,ratio_h = point_to_coord_boundry(bbox,bbx_coord)
    #get the conversion 
    #coord = point min + (point-point min) * ratio
    lat = []
    long = []
    for index, row in df.iterrows():
        x = (row['lat'] - bbox[0]) * ratio_len
        y = (row['long'] - bbox[1]) * ratio_h
        #print(x)
        ##can't do like this
        lat.append(bbx_coord[0]+x)
        long.append(bbx_coord[3]-y) #since the image location mapping difference with bbox coord
        
    poly = list(zip(lat, long))
    return poly

In [12]:
greens,white = get_contours(img,hsv)

In [13]:
boundry_poly,bbox = point_to_coord_boundry(white,bbx_coord)
boundry_poly

[(-86.69479747895048, 50.18042402227904),
 (-86.69475929403134, 50.18044848379505),
 (-86.69353737661895, 50.18044848379505),
 (-86.69349919169981, 50.18042402227904),
 (-86.69349919169981, 50.180326176215004),
 (-86.69330826710413, 50.180326176215004),
 (-86.693270082185, 50.18030171469899),
 (-86.693270082185, 50.18017940711895),
 (-86.69311734250844, 50.18017940711895),
 (-86.69307915758931, 50.180154945602936),
 (-86.69307915758931, 50.18005709953891),
 (-86.69288823299362, 50.18005709953891),
 (-86.69285004807448, 50.180032638022894),
 (-86.69285004807448, 50.17991033044285),
 (-86.69269730839794, 50.17991033044285),
 (-86.69265912347879, 50.17988586892684),
 (-86.69265912347879, 50.1797880228628),
 (-86.6924681988831, 50.1797880228628),
 (-86.69243001396397, 50.179763561346796),
 (-86.69243001396397, 50.179518946186704),
 (-86.69227727428742, 50.179518946186704),
 (-86.69223908936829, 50.1794944846707),
 (-86.69223908936829, 50.179372177090656),
 (-86.6920481647726, 50.1793721770

In [14]:
greens = sorted(greens, key=cv2.contourArea, reverse= True)

In [15]:
green_coords = []
for green in greens:
    #print(green.shape)
    g_coord = point_to_coord_green(green,bbx_coord,bbox)
    green_coords.append(g_coord)

In [16]:
green_coords

[[(-86.69475929403134, 50.18042402227904),
  (-86.69475929403134, 50.180326176215004),
  (-86.69479747895048, 50.18030171469899),
  (-86.69559936225237, 50.18030171469899),
  (-86.69559936225237, 50.18017940711895),
  (-86.6956375471715, 50.180154945602936),
  (-86.69579028684805, 50.180154945602936),
  (-86.69579028684805, 50.18005709953891),
  (-86.69582847176719, 50.180032638022894),
  (-86.69601939636287, 50.180032638022894),
  (-86.696057581282, 50.18005709953891),
  (-86.696057581282, 50.180154945602936),
  (-86.69621032095856, 50.180154945602936),
  (-86.69621032095856, 50.18005709953891),
  (-86.69624850587769, 50.180032638022894),
  (-86.6972794986944, 50.180032638022894),
  (-86.6972794986944, 50.179372177090656),
  (-86.69731768361353, 50.17934771557464),
  (-86.69831049151111, 50.17934771557464),
  (-86.69834867643024, 50.179372177090656),
  (-86.69834867643024, 50.1794944846707),
  (-86.69853960102593, 50.1794944846707),
  (-86.69857778594506, 50.179518946186704),
  (-86.6

In [17]:
from Utils.map_view import map_show

In [18]:
map_show(boundry_poly)

In [18]:
greens[0].shape

(427, 1, 2)

In [20]:
map_show(green_coords[1])